beta = [0,2]    alpha = 0.333
init_beta = 1
mean_beta = 0.85

without fisher:
    acc : 85.52 --> 64.03
    drop : 21.49

with fisher:
    acc : 85.52 --> 35.12
    drop : 50.4

In [1]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import sys
import numpy as np
import pickle

sys.path.append('D:\\Program\\MyCode\\Round_robin_SL\\Round-Robin')
from models import *
from clients_datasets import *
from tqdm.notebook import tqdm
from utils import *
from AttFunc import *
from Fisher_LeNet import *

In [2]:
batch_size = 600
epochs = 30
NC = 10
dataset = 'cifar10'

clients_trainloader = load_clients_trainsets(dataset, NC, batch_size)
clients_testloader = load_clients_testsets(dataset, NC, batch_size)

server, server_opt, clients, clients_opts = set_model_and_opt(dataset, NC)
client_level = 1
server_level = 6

criterion = torch.nn.CrossEntropyLoss()

Files already downloaded and verified
Subset 1: Class counts - [515, 510, 533, 487, 507, 519, 489, 459, 489, 492]
Subset 2: Class counts - [481, 486, 505, 501, 461, 526, 503, 506, 501, 530]
Subset 3: Class counts - [480, 486, 490, 538, 474, 523, 531, 498, 526, 454]
Subset 4: Class counts - [525, 465, 501, 510, 550, 489, 491, 473, 471, 525]
Subset 5: Class counts - [537, 501, 472, 517, 529, 481, 509, 494, 506, 454]
Subset 6: Class counts - [462, 519, 498, 490, 535, 515, 471, 488, 501, 521]
Subset 7: Class counts - [477, 499, 507, 473, 504, 474, 521, 551, 501, 493]
Subset 8: Class counts - [494, 481, 506, 500, 501, 502, 495, 490, 518, 513]
Subset 9: Class counts - [496, 540, 521, 484, 444, 462, 506, 520, 520, 507]
Subset 10: Class counts - [533, 513, 467, 500, 495, 509, 484, 521, 467, 511]
Files already downloaded and verified
Subset 1: Class counts - [87, 100, 108, 107, 95, 95, 100, 102, 102, 104]
Subset 2: Class counts - [107, 108, 100, 85, 95, 99, 116, 97, 99, 94]
Subset 3: Class coun

In [6]:
clients_acc0 = []
#################################################################################
###########################   Normal training   #################################
# train
server.train()
for i in range(NC):
    clients[i].train()
server.apply(init_weights)
clients[0].apply(init_weights)
last_trained_params = clients[0].state_dict()
for epoch in tqdm(range(epochs), desc="Normal Training", unit="eopch"):
# for epoch in range(epochs):
    for idx, client in enumerate(clients):
        client.load_state_dict(last_trained_params)
        for j, data in enumerate(clients_trainloader[idx]):
            images, labels = data
            images = images.cuda()
            labels = labels.cuda()
            smashed_data = client.forward(images, client_level=client_level)
            output = server.forward(smashed_data, server_level=server_level)
            clients_opts[idx].zero_grad()
            server_opt.zero_grad()
            loss = criterion(output, labels)
            loss.backward()
            clients_opts[idx].step()
            server_opt.step()
        last_trained_params = client.state_dict()
for i in range(NC):
    clients[i].load_state_dict(last_trained_params)
# test
server.eval()
for i in range(NC):
    clients[i].eval()
with torch.no_grad():
    for idx, client in enumerate(tqdm(clients)):
        correct = 0
        total = 0
        acc0 = 0
        for data in clients_testloader[idx]:
            images, labels = data
            images, labels = images.cuda(), labels.cuda()

            smashed_data = client.forward(images, client_level=client_level)
            output = server.forward(smashed_data, server_level=server_level)
            _, pre = torch.max(output.data, 1)
            total += images.shape[0]
            correct += (pre == labels).sum().item()
        acc0 = 100 * correct / total
        clients_acc0.append(acc0)
acc0 = np.mean(clients_acc0)
print(acc0)

Normal Training:   0%|          | 0/30 [00:00<?, ?eopch/s]

  0%|          | 0/10 [00:00<?, ?it/s]

85.52000000000001


In [3]:
am = ResNet_9().cuda()
am_train(am, clients_trainloader[8], dataset)
fisher_matrix = {}
for param_name, param in am.named_parameters():
    if param_name == 'conv1.0.weight':
        grad = param.grad.cpu().detach().numpy()
        if param_name not in fisher_matrix:
            fisher_matrix[param_name] = grad ** 2
        else:
            fisher_matrix[param_name] += grad ** 2
    if param_name == 'conv1.0.bias':
        grad = param.grad.cpu().detach().numpy()
        if param_name not in fisher_matrix:
            fisher_matrix[param_name] = grad ** 2
        else:
            fisher_matrix[param_name] += grad ** 2

Training attack model:   0%|          | 0/100 [00:00<?, ?eopch/s]

In [4]:
am_params = torch.load(f'../Round_robin_SL/Round-Robin/params/am_{dataset}_params.pth')
am_acc0 = am_test(am, am_params, clients_testloader[8], 0)
print(-am_acc0)

64.9


In [5]:
att_type = 'unit'
acc0 = 85.52
betas = []

for i in tqdm(range(10), desc="Finding beta", unit="iter"):
    beta = simulated_annealing(dataset, 1, 500, att_type, acc0,  clients_testloader[8])
    betas.append(beta)
    print(beta)

Finding beta:   0%|          | 0/10 [00:00<?, ?iter/s]

0.7521691175988757
0.9541863840455063
0.5624994921688379
1.3830029336956007
1.2021923389375215
0.7169582311977346
0.8057095079528475
0.7765631941399591
0.5566204668606384
0.9789293410835386


In [6]:
beta_mean, beta_range = data_process(betas)
print(beta_mean)

0.8535115089641454


---------------------------------------------------------
## without fisher

In [7]:
acc0 = 85.52
att_type = 'unit'
acc1_varying = []
drop_varying = []
clients_acc1 = []
clients_drop = []
iters = 5


for iter in tqdm(range(iters), desc="Training", unit="iter"):
    batch_size = 600
    epochs = 30
    NC = 10
    dataset = 'cifar10'

    clients_trainloader = load_clients_trainsets(dataset, NC, batch_size)
    clients_testloader = load_clients_testsets(dataset, NC, batch_size)

    server, server_opt, clients, clients_opts = set_model_and_opt(dataset, NC)
    client_level = 1
    server_level = 6

    criterion = torch.nn.CrossEntropyLoss()
    # train
    mal_client_id = [8]
    server.train()
    for i in range(NC):
        clients[i].train()
    server.apply(init_weights)
    clients[0].apply(init_weights)
    last_trained_params = clients[0].state_dict()
    for epoch in range(epochs):
        beta = beta_mean
        for idx, client in enumerate(clients):
            client.load_state_dict(last_trained_params)
            for j, data in enumerate(clients_trainloader[idx]):
                # training part
                images, labels = data
                images = images.cuda()
                labels = labels.cuda()
                smashed_data = client.forward(images, client_level=client_level)
                output = server.forward(smashed_data, server_level=server_level)
                clients_opts[idx].zero_grad()
                server_opt.zero_grad()
                loss = criterion(output, labels)
                loss.backward()
                clients_opts[idx].step()
                server_opt.step()
            # weight sharing
            last_trained_params = client.state_dict()
            # attack part
            if idx in mal_client_id :
                benign_params = list(client.parameters())[:2]
                mal_params = perturbation(beta=beta, bp=benign_params, type=att_type)
                last_trained_params['conv1.0.weight'] = mal_params[0]
                last_trained_params['conv1.0.bias'] = mal_params[1]
    for i in range(NC):
        clients[i].load_state_dict(last_trained_params)

    # test
    server.eval()
    for i in range(NC):
        clients[i].eval()
    with torch.no_grad():
        for idx, client in enumerate(clients):
            correct = 0
            total = 0
            acc1 = 0
            for data in clients_testloader[idx]:
                images, labels = data
                images, labels = images.cuda(), labels.cuda()

                smashed_data = client.forward(images, client_level=client_level)
                output = server.forward(smashed_data, server_level=server_level)
                _, pre = torch.max(output.data, 1)
                total += images.shape[0]
                correct += (pre == labels).sum().item()
            acc1 = 100 * correct / total
            clients_acc1.append(acc1)
            drop = acc0 - acc1
            clients_drop.append(drop)
    acc1 = np.mean(clients_acc1)
    acc1_varying.append(acc1)
    drop = np.mean(clients_drop)
    drop_varying.append(drop)

Training:   0%|          | 0/5 [00:00<?, ?iter/s]

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [9]:
print(acc1_varying)
print(np.mean(acc1_varying))
print('----------------------------------------------------')
print(drop_varying)
print(np.mean(drop_varying))

[77.68, 78.55000000000001, 79.38333333333331, 76.7225, 77.50200000000001]
[7.839999999999998, 6.969999999999996, 6.136666666666664, 8.797499999999996, 8.017999999999997]
[77.68, 78.55000000000001, 79.38333333333331, 76.7225, 77.50200000000001]
77.96756666666667
----------------------------------------------------
[7.839999999999998, 6.969999999999996, 6.136666666666664, 8.797499999999996, 8.017999999999997]
7.5524333333333304


In [18]:
acc1_mean, acc1_range = data_process(acc1_varying)
print('acc1:')
print(acc1_mean)
print(acc1_range)
print('---------------------')
drop_mean, drop_range = data_process(drop_varying)
print('drop:')
print(drop_mean)
print(drop_range)

acc1:
64.03088888888888
5.2933333333333294
---------------------
drop:
21.489111111111104
5.2933333333333294


-------------------------------------------
## with fisher

In [10]:
weight_positions = []
bias_positions = []
weight_positions.append(find_positions(fisher_matrix['conv1.0.weight'], 0.333))
bias_positions.append(find_positions(fisher_matrix['conv1.0.bias'], 0.333))

In [11]:
acc0 = 85.52
att_type = 'unit'
acc1_varying = []
drop_varying = []
clients_acc1 = []
clients_drop = []
iters = 5


for iter in tqdm(range(iters), desc="Training", unit="iter"):
    batch_size = 600
    epochs = 30
    NC = 10
    dataset = 'cifar10'

    clients_trainloader = load_clients_trainsets(dataset, NC, batch_size)
    clients_testloader = load_clients_testsets(dataset, NC, batch_size)

    server, server_opt, clients, clients_opts = set_model_and_opt(dataset, NC)
    client_level = 1
    server_level = 6

    criterion = torch.nn.CrossEntropyLoss()
    # train
    mal_client_id = [8]
    server.train()
    for i in range(NC):
        clients[i].train()
    server.apply(init_weights)
    clients[0].apply(init_weights)
    last_trained_params = clients[0].state_dict()
    for epoch in range(epochs):
        beta = beta_mean
        for idx, client in enumerate(clients):
            client.load_state_dict(last_trained_params)
            for j, data in enumerate(clients_trainloader[idx]):
                # training part
                images, labels = data
                images = images.cuda()
                labels = labels.cuda()
                smashed_data = client.forward(images, client_level=client_level)
                output = server.forward(smashed_data, server_level=server_level)
                clients_opts[idx].zero_grad()
                server_opt.zero_grad()
                loss = criterion(output, labels)
                loss.backward()
                clients_opts[idx].step()
                server_opt.step()
            # weight sharing
            last_trained_params = client.state_dict()
            # attack part
            if idx in mal_client_id :
                benign_params = list(client.parameters())[:2]
                mal_params = fisher_perturbation(client_level, beta, benign_params, weight_positions, bias_positions, type=att_type)
                last_trained_params['conv1.0.weight'] = mal_params[0]
                last_trained_params['conv1.0.bias'] = mal_params[1]
    for i in range(NC):
        clients[i].load_state_dict(last_trained_params)

    # test
    server.eval()
    for i in range(NC):
        clients[i].eval()
    with torch.no_grad():
        for idx, client in enumerate(clients):
            correct = 0
            total = 0
            acc1 = 0
            for data in clients_testloader[idx]:
                images, labels = data
                images, labels = images.cuda(), labels.cuda()

                smashed_data = client.forward(images, client_level=client_level)
                output = server.forward(smashed_data, server_level=server_level)
                _, pre = torch.max(output.data, 1)
                total += images.shape[0]
                correct += (pre == labels).sum().item()
            acc1 = 100 * correct / total
            clients_acc1.append(acc1)
            drop = acc0 - acc1
            clients_drop.append(drop)
    acc1 = np.mean(clients_acc1)
    acc1_varying.append(acc1)
    drop = np.mean(clients_drop)
    drop_varying.append(drop)

Training:   0%|          | 0/5 [00:00<?, ?iter/s]

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [13]:
print(acc1_varying)
print(drop_varying)

[10.000000000000002, 10.0, 35.120000000000005, 47.68249999999999, 55.275999999999996]
[75.51999999999998, 75.52, 50.4, 37.8375, 30.243999999999996]


In [12]:
acc1_mean, acc1_range = data_process(acc1_varying)
print('acc1:')
print(acc1_mean)
print(acc1_range)
print('---------------------')
drop_mean, drop_range = data_process(drop_varying)
print('drop:')
print(drop_mean)
print(drop_range)

acc1:
35.120000000000005
0.0
---------------------
drop:
50.4
0.0
